In [177]:
import pandas as pd
ted_train=pd.read_csv("/home/nxingyu/data/ted_talks_processed.train.csv")
ted_dev=pd.read_csv("/home/nxingyu/data/ted_talks_processed.dev.csv")
ted_test=pd.read_csv("/home/nxingyu/data/ted_talks_processed.test.csv")

In [107]:
ted_train.transcript[ted_train.transcript.apply(lambda x:len(x.split()))==12]#.describe()

131     Thank you. Marcus Miller. Harvey Mason. Thank...
Name: transcript, dtype: object

In [130]:
open=pd.read_csv("/home/nxingyu/data/open_subtitles.csv")

In [132]:
open_test=pd.read_csv("/home/nxingyu/data/open_subtitles_processed.test.csv")
open_dev=pd.read_csv("/home/nxingyu/data/open_subtitles_processed.dev.csv")
open_train=pd.read_csv("/home/nxingyu/data/open_subtitles_processed.train.csv")

In [15]:
open_test.transcript[open_test.transcript.apply(lambda x:len(x.split()))==17]#.describe()

1320    THOSE LOVE PANGS RIVALS THERE'S A FEMALE HIDIN...
Name: transcript, dtype: object

In [133]:
def punct_proportion(df):
    l=[]
    for c in ".?!,;:-—":
        l.append(sum(df.transcript.str.count("\\"+c)))
    [print(i[0],i[1],i[1]/sum(l)) for i in zip(list(".?!,;:-—"),l)]
for split in open_train, open_dev, open_test:
# for split in ted_train,ted_dev,ted_test:
    punct_proportion(split)
    print()

. 2551690 0.4508845443374779
? 687648 0.12150764988873178
! 361404 0.06386021729196802
, 1353949 0.23924327716971258
; 4450 0.0007863166067593543
: 18042 0.00318802791441624
- 680907 0.12031651275476217
— 1208 0.00021345403617197752

. 339576 0.45784149421118553
? 88661 0.11953932173727802
! 45729 0.06165522206747033
, 177440 0.23923773980738558
; 213 0.00028718236349736884
: 4321 0.005825891984376201
- 85308 0.11501855899170677
— 441 0.0005945888371001862

. 333653 0.4527804390838352
? 88275 0.1197926985824361
! 45666 0.06197058480278139
, 176580 0.23962610836235138
; 293 0.0003976126953798219
: 1929 0.0026177299979101586
- 90152 0.12233986250471571
— 350 0.0004749639705902309



In [178]:
from datasets import load_dataset
ted=load_dataset('csv',data_files={'train':'/home/nxingyu/data/ted_talks_processed.train.csv',
                                         'dev':'/home/nxingyu/data/ted_talks_processed.dev.csv',
                                         'test':'/home/nxingyu/data/ted_talks_processed.test.csv'})
subtitles=load_dataset('csv',data_files={'train':'/home/nxingyu/data/open_subtitles_processed.train.csv',
                                         'dev':'/home/nxingyu/data/open_subtitles_processed.dev.csv',
                                         'test':'/home/nxingyu/data/open_subtitles_processed.test.csv'})

Using custom data configuration default
Reusing dataset csv (/home/nxingyu/.cache/huggingface/datasets/csv/default-f6b55550f42cdbb1/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)
Using custom data configuration default
Reusing dataset csv (/home/nxingyu/.cache/huggingface/datasets/csv/default-ac279971daecd65a/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


In [129]:
ted_train.sort_values('talk_id').sample(frac=1,random_state=42).reset_index(drop=True)

,talk_id,transcript
0,1561,"Some years ago, I set out to try to understand..."
1,1526,I thought if I skipped it might help my nerves...
2,1215,Today I'd like to show you the future of the w...
3,633,"These are grim economic times, fellow TEDsters..."
4,1655,Let me tell you a story. It's my first year as...
...,...,...
3184,1644,"Hello. My name is Jarrett Krosoczka, and I wri..."
3185,1692,I have a friend in Portugal whose grandfather ...
3186,1910,Some of my most wonderful memories of childhoo...
3187,1324,Nearly everyone in the world is part of some c...


In [134]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [45]:
from tokenizers.processors import BertProcessing


In [179]:
tags=sorted(list('.?!,;:-—…'))
tag2id = {tag: id+1 for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
tag2id,id2tag

({'!': 1, ',': 2, '-': 3, '.': 4, ':': 5, ';': 6, '?': 7, '—': 8, '…': 9},
 {1: '!', 2: ',', 3: '-', 4: '.', 5: ':', 6: ';', 7: '?', 8: '—', 9: '…'})

In [180]:
tokens=tokenizer.tokenize(ted['train'][0]['transcript'])
len(list(map(lambda x:tag2id[x] if x in tags else 0,tokens))),len(tokens)

(3068, 3068)

In [181]:
# from nltk import word_tokenize
# nltk.download('punkt')
# transcript=word_tokenize(ted['train'][0]['transcript'])
# import regex as re
# re.split(r'[.?!,;:\-—…] ?| +',ted['train'][0]['transcript'])
# for c in '.?!,;:-—',
# v=transcript.split()
# [i for j, i in range(len(v)*2-1) if j % 2] 
ted['train'][0]['transcript']

"One summer afternoon in 2013, DC police detained, questioned and searched a man who appeared suspicious and potentially dangerous. This wasn't what I was wearing the day of the detention, to be fair, but I have a picture of that as well. I know it's very frightening — try to remain calm. At this time, I was interning at the Public Defender Service in Washington DC, and I was visiting a police station for work. I was on my way out, and before I could make it to my car, two police cars pulled up to block my exit, and an officer approached me from behind. He told me to stop, take my backpack off and put my hands on the police car parked next to us. About a dozen officers then gathered near us. All of them had handguns, some had assault rifles. They rifled through my backpack. They patted me down. They took pictures of me spread on the police car, and they laughed. And as all this was happening — as I was on the police car trying to ignore the shaking in my legs, trying to think clearly a

In [213]:
def text2masks(text,n):
    refilter="(?<=[.?!,;:\-—… ])(?=[^.?!,;:\-—… ])|$" if n==0 else "[.?!,;:\-—…]{1,%d}(?= *[^.?!,;:\-—…]+|$)|(?<=[^.?!,;:\-—…]) +(?=[^.?!,;:\-—…])"%(n)
    punct=re.findall(refilter,text)
    word=re.split(refilter,text)
    wordlist=[]
    punctlist=[]
    for i in zip(word,punct+['']):
        w,p=i[0].strip(),i[1].strip()
        if w!='':
            wordlist.append(re.sub(r'[.?!,;:\-—… ]','',w))
            punctlist.append(0 if not w[-1] in '.?!,;:-—…' else tag2id[w[-1]])
        if p!='':
            wordlist.append(p)
            punctlist.append(0)
    return(wordlist,punctlist)
text,tag=text2masks(ted['train'][0]['transcript'],0)
texts=[]
tags=[]
for i in range(len(text)//15):
    texts.append(text[i*15:min((i+1)*15,len(text))])
    tags.append(tag[i*15:min((i+1)*15,len(text))])

In [137]:
text,tag=text2masks(ted['train'][0]['transcript'],0)
encoding=tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [217]:
encodings = tokenizer(texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
encodings

{'input_ids': [[101, 1448, 2247, 4427, 1107, 1381, 5227, 2021, 15474, 8449, 1105, 8703, 170, 1299, 1150, 1691, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10108, 1105, 9046, 4249, 1188, 1445, 112, 189, 1184, 146, 1108, 3351, 1103, 1285, 1104, 1103, 13826, 102, 0, 0, 0, 0, 0, 0], [101, 1106, 1129, 4652, 1133, 146, 1138, 170, 3439, 1104, 1115, 1112, 1218, 146, 1221, 1122, 112, 188, 102, 0, 0, 0, 0, 0, 0], [101, 1304, 18523, 2222, 1106, 3118, 5017, 1335, 1142, 1159, 146, 1108, 9455, 3381, 1120, 1103, 2710, 102, 0, 0, 0, 0, 0, 0, 0], [101, 3177, 27896, 2516, 1107, 1994, 5227, 1105, 146, 1108, 5807, 170, 2021, 1466, 1111, 1250, 146, 102, 0, 0, 0, 0, 0, 0, 0], [101, 1108, 1113, 1139, 1236, 1149, 1105, 1196, 146, 1180, 1294, 1122, 1106, 1139, 1610, 1160, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2021, 3079, 1865, 1146, 1106, 3510, 1139, 6300, 1105, 1126, 2575, 4685, 1143, 1121, 1481, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1124, 1500, 1143, 1106, 1831, 1321, 1139, 13846, 1228, 1105, 1508, 1139, 1493, 1113, 110

In [220]:
def encode_tags(encodings, labels):
    encoded_labels = []
    for doc_labels, doc_offset in zip(tags, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

labels=encode_tags(encodings, tags)

In [223]:
import tensorflow as tf
encodings.pop("offset_mapping")

dataset = tf.data.Dataset.from_tensor_slices((
    dict(encodings),
    labels
))

In [227]:
for data in dataset.take(1):
    print(data)

({'input_ids': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([  101,  1448,  2247,  4427,  1107,  1381,  5227,  2021, 15474,
        8449,  1105,  8703,   170,  1299,  1150,  1691,   102,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)>}, <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([-100,    0,    0,    0,    0,    2,    0,    0,    2,    0,    0,
          0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100,
       -100, -100, -100], dtype=int32)>)


In [ ]:
# from transformers import TFBertForSequenceClassification
# model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=len(unique_tags))

In [57]:
tokenizer(ted['train'][0]['transcript'], return_offsets_mapping=True, padding=True, truncation=True)

{'input_ids': [101, 20062, 117, 1166, 122, 1553, 126, 3775, 1234, 2541, 4223, 4139, 119, 1130, 2593, 117, 1234, 1132, 2257, 1106, 10556, 1147, 1583, 117, 2128, 1166, 1405, 1550, 8940, 119, 4288, 117, 1443, 170, 4095, 117, 1132, 1103, 1211, 7386, 1105, 8018, 5256, 795, 1133, 1136, 1198, 1121, 1103, 5119, 2952, 18552, 117, 1133, 1121, 1103, 1510, 8362, 20080, 27443, 3154, 1115, 8755, 1138, 1113, 1147, 2073, 119, 1109, 5758, 1104, 1594, 1817, 1482, 1120, 170, 1842, 1344, 3187, 1111, 1103, 1718, 1104, 6438, 1105, 18560, 2645, 119, 4288, 117, 1112, 1195, 1169, 1178, 5403, 117, 1209, 1631, 4472, 117, 4963, 1105, 1120, 3187, 119, 1252, 1175, 1110, 1363, 2371, 119, 1109, 3068, 1104, 1920, 1115, 1482, 3531, 1107, 1147, 2073, 1169, 1138, 170, 1167, 2418, 2629, 1113, 1147, 1218, 118, 1217, 1190, 1121, 1103, 4315, 5758, 1104, 1594, 1115, 1152, 1138, 1151, 5490, 1106, 119, 1573, 2140, 117, 1482, 1169, 1129, 4921, 1118, 3258, 117, 5343, 6486, 1158, 1219, 1105, 1170, 4139, 119, 1130, 1349, 117, 146, 

In [50]:
#create_pretraining_data.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import random
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
import tensorflow as tf

flags = tf.compat.v1.flags
FLAGS = flags.FLAGS

In [51]:
class TrainingInstance(object):
    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels, is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

<module 'tensorflow.python.platform.flags' from '/home/nxingyu/miniconda3/envs/NLP/lib/python3.8/site-packages/tensorflow/python/platform/flags.py'>